## Import Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys
from scipy.stats import zscore
import time

sns.set()

# Add path to root directory with data
sys.path += ['./data']

## Constants

In [2]:
# Data Paths
TRAIN_PATH      = sys.path[-1] + '/train.csv'
GRAPH_PATH      = sys.path[-1] + '/graph.csv'
TEST_PATH       = sys.path[-1] + '/test.csv'
MERGED_PATH     = sys.path[-1] + '/merged_raw.csv'
SUBMISSION_PATH = sys.path[-1] + '/submission.csv'

# Kaggle Baseline Score
BASELINE_MAPE = 2.93299

# Set to True if you want to merge dataframes from scratch
MERGE_DATAFRAMES = False

## Helper Functions

In [3]:
def display_column_values(data_frame, col_name):
    '''
        View the distribution of data within a particular column
        in a given dataframe
    '''
    return data_frame[col_name].value_counts()


def count_NaNs(data_frame, col_name):
    '''
        Return the number of NaN values in a particular column for 
        a given dataframe
    '''
    return data_frame[col_name].isnull().sum()


def calculate_NaN_replacement(data_frame, col_name, state, strategy=None):
    '''
        Helper for fill_NaNs
        Different strategies for filling NaN values (default = 0)
        - Average: Return mean of the column for the state
        - Median: Return the median of the column for the state
    '''
    if strategy=='mean' or strategy=='average':
        return data_frame[data_frame.Province_State == state][col_name].mean()
    elif strategy=='median':
        return data_frame[data_frame.Province_State == state][col_name].mean()
    else:
        return 0


def calculate_moving_averages(data_frame, state, col_name, n):
    '''
        Returns a Pandas Series of computed moving averages for a given column
        Inputs:
            data_frame: merged_df
            state: state name
            col_name: feature to calculate moving average for
            n: number of days we want to calculate the moving average for
    '''
    # Get last n days of data from dataframe for that particular column
    subset = list(data_frame[data_frame.Province_State == state][col_name][-n:])
    subset += [0]*26
    for i in range(n, len(subset)):
        subset[i] = np.array(subset[i-n:i]).mean()
    subset = subset[n:]
    return subset


def one_hot_encode(dataframe, feature):
    '''
    Returns one hot encoded 2D vector
    '''
    n, d = dataframe.shape
    one_hot_vector = np.zeros((n, len(dataframe[feature].unique())))
    # Create mapping
    mapping = dict()
    for idx, val in enumerate(dataframe[feature].unique()):
        mapping[val] = idx
    # Create one hot vector
    for index, row in dataframe.iterrows():
        one_hot_vector[index][mapping[row[feature]]] = 1
    return one_hot_vector


def normalize(dataframe, feature):
    '''
    Takes an ordinal feature and normalizes its range to between [0,1],
    where 0 is the min value and 1 corresponds to the max value.
    '''
    max_ = max(dataframe[feature])
    min_ = min(dataframe[feature])
    normalized = dataframe[feature].apply(lambda x: (x-min_)/(max_-min_))
    return normalized


def percentage_error(actual, predicted):
    '''
        Helper method for MAPE. Returns percentage error between
        predicted values and ground truth values.
    '''
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res


def MAPE(y_true, y_pred):
    '''
        Inputs:
            - y_true: ground truth values
            - y_pred: predicted values
        Returns:
            - Mean Absolute Percentage Error (MAPE), a value that lies in the range [0,100]
    '''
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100

## Import Data

In [4]:
train = pd.read_csv(TRAIN_PATH)
graph = pd.read_csv(GRAPH_PATH)
test  = pd.read_csv(TEST_PATH)

## Data Processing

In [5]:
# Drop Id row because it has 7100 unique values, the same as the number of rows in the dataset
train.drop(['ID'], inplace=True, axis=1)

#### Create training data sorted by state and date

In [6]:
# Get all state names
state_names = train['Province_State'].unique()

# Group training data by state names
group_by_state = train.groupby(train.Province_State)

# Create training data sorted by state and date
training_grouped_by_state = pd.DataFrame()
for state in state_names:
    state_specific_df = pd.DataFrame(group_by_state.get_group(state))
    training_grouped_by_state = training_grouped_by_state.append(state_specific_df)

# Reset index since group by creates incorrect indexes
training_grouped_by_state = training_grouped_by_state.reset_index(drop=True)

training_grouped_by_state.head(3)

,Province_State,Date,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
0,Alabama,04-12-2020,3563,93,NaN,3470.0,75.988020,21583.0,437.0,2.610160,460.300152,12.264945
1,Alabama,04-13-2020,3734,99,NaN,3635.0,79.634933,29182.0,457.0,2.651312,622.363852,12.238886
2,Alabama,04-14-2020,3953,114,NaN,3839.0,84.305541,33117.0,493.0,2.883886,706.285508,12.471541


#### Count NaN values for the 3 features with NaN values

If we do decide to proceed with a state specific models, we can use this data to create custom datasets for each state depending on the number of NaN values. 

In [7]:
total_number_of_days = len(graph.columns[2:])
for state in state_names:
    state_specific_data = training_grouped_by_state[training_grouped_by_state.Province_State == state]
    print("State: {}".format(state))
    print("\t-- Recovered (NaNs): {}/{}".format(count_NaNs(state_specific_data, 'Recovered'), total_number_of_days))
    print("\t-- People Hospitalized (NaNs): {}/{}".format(count_NaNs(state_specific_data, 'People_Hospitalized'), total_number_of_days))
    print("\t-- Hospitalization Rate (NaNs): {}/{}".format(count_NaNs(state_specific_data, 'Hospitalization_Rate'), total_number_of_days))
    print()

State: Alabama
	-- Recovered (NaNs): 35/142
	-- People Hospitalized (NaNs): 4/142
	-- Hospitalization Rate (NaNs): 4/142

State: Alaska
	-- Recovered (NaNs): 0/142
	-- People Hospitalized (NaNs): 125/142
	-- Hospitalization Rate (NaNs): 125/142

State: Arizona
	-- Recovered (NaNs): 2/142
	-- People Hospitalized (NaNs): 5/142
	-- Hospitalization Rate (NaNs): 5/142

State: Arkansas
	-- Recovered (NaNs): 0/142
	-- People Hospitalized (NaNs): 5/142
	-- Hospitalization Rate (NaNs): 5/142

State: California
	-- Recovered (NaNs): 137/142
	-- People Hospitalized (NaNs): 126/142
	-- Hospitalization Rate (NaNs): 126/142

State: Colorado
	-- Recovered (NaNs): 13/142
	-- People Hospitalized (NaNs): 4/142
	-- Hospitalization Rate (NaNs): 4/142

State: Connecticut
	-- Recovered (NaNs): 18/142
	-- People Hospitalized (NaNs): 7/142
	-- Hospitalization Rate (NaNs): 7/142

State: Delaware
	-- Recovered (NaNs): 0/142
	-- People Hospitalized (NaNs): 126/142
	-- Hospitalization Rate (NaNs): 126/142

State:

#### Merge Datasets

In [8]:
if MERGE_DATAFRAMES:
    # Create a new column for each state in the dataset
    for state in state_names:
        training_grouped_by_state[state] = 0
    print("New number of columns:", training_grouped_by_state.shape[1])
    # Get list of all the dates
    all_dates = graph.columns[2:]
    # Initialize empty dataframe
    merged_df = pd.DataFrame()
    start = time.time()
    # Try and merge these two dataframes together
    for target_state in state_names:
        # Make sure target states are the same
        state_train_data = training_grouped_by_state[training_grouped_by_state.Province_State == target_state]
        state_graph_data = graph[graph.target_state == target_state]
        # Loop through all the source_states and dates
        for source_state in state_names:
            for date in all_dates:
                row = state_train_data[state_train_data.Date == date]
                row_number = row.index[0]
                # Update value
                state_train_data.at[row_number, source_state] = state_graph_data[state_graph_data.source_state == source_state][date]
        # Append data for this target state to the created dataframe
        merged_df = merged_df.append(state_train_data)
        print("Completed: ", target_state)
    end = time.time()
    print("Total merge time: ", end-start)
    # Save Merged DataFrame
    merged_df.to_csv('./data/merged_raw.csv', index=False)
else:
    merged_df = pd.read_csv(MERGED_PATH)

#### Fill NaN values with 0s for that particular column [Done on a per-state basis]

In [9]:
# CAN IGNORE SET WITH COPY WARNING
temp_df = pd.DataFrame()
for state in state_names:
    # Get mean values for Recovered, People Hospitalized and Hospitalization Rate
    mean_recovered = calculate_NaN_replacement(merged_df, 'Recovered', state, None)
    mean_hospitalizations = calculate_NaN_replacement(merged_df, 'People_Hospitalized', state, None)
    mean_hospitalization_rate = calculate_NaN_replacement(merged_df, 'Hospitalization_Rate', state, None)
    # Get Slice
    state_df = merged_df[merged_df.Province_State == state]
    # Update values
    state_df['Recovered'].fillna(mean_recovered, inplace=True)
    state_df['People_Hospitalized'].fillna(mean_hospitalizations, inplace=True)
    state_df['Hospitalization_Rate'].fillna(mean_hospitalization_rate, inplace=True)
    # Add to temp df
    temp_df = temp_df.append(state_df)
merged_df = temp_df

/usr/local/lib/python3.7/site-packages/pandas/core/series.py:4535: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


# [Break] Calculate Moving Average For Test Dataset

In [10]:
# CAN IGNORE SET WITH COPY WARNING

print("Columns:\n", list(merged_df.columns))

# Drop columns that we don't want to calculate moving averages for
features = list(merged_df.columns)[4:]

temp_df = pd.DataFrame()
for state in state_names:
    # Get slice of data for that specific state
    state_df = test[test.Province_State == state]
    # Compute 7 day moving average for each feature
    for feature in features:
        state_df[feature] = calculate_moving_averages(merged_df, state, feature, 7)
    temp_df = temp_df.append(state_df)
    
temp_df = temp_df.sort_values(by='ForecastID')
test = temp_df

test.head(3)

Columns:
 ['Province_State', 'Date', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Incident_Rate', 'People_Tested', 'People_Hospitalized', 'Mortality_Rate', 'Testing_Rate', 'Hospitalization_Rate', 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ForecastID,Province_State,Date,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,0,Alabama,09-01-2020,-1,-1,47550.285714,72467.571429,2490.760015,9.621144e+05,5971.857143,...,48.428571,7150.000000,3141.571429,83.571429,22.428571,703.142857,167.857143,159.285714,228.714286,33.142857
1,1,Alaska,09-01-2020,-1,-1,2145.571429,2885.428571,692.760820,3.407939e+05,0.000000,...,12.428571,55.571429,301.571429,80.714286,2.142857,50.714286,463.142857,7.285714,70.857143,20.428571
2,2,Arizona,09-01-2020,-1,-1,30124.857143,165539.571429,2756.200028,1.183481e+06,9166.714286,...,136.571429,403.428571,3026.571429,1249.714286,3.714286,367.857143,1397.285714,35.857143,519.000000,137.571429


In [16]:
# Save test DataFrame
test.to_csv('./data/test_raw.csv', index=False)

# [Resume] Data Processing

#### Z-Score Normalization

In [11]:
non_normalized_columns = merged_df[['Province_State', 'Date', 'Confirmed', 'Deaths']]
merged_df.drop(['Province_State','Date','Confirmed', 'Deaths'], inplace=True, axis=1)

# Perform Z-Score normalization
merged_df_cols = merged_df.columns
normalized_data = zscore(merged_df)
merged_df = pd.DataFrame(data=normalized_data, columns=merged_df_cols)

# Reassign Columns
merged_df['Province_State'] = non_normalized_columns['Province_State']
merged_df['Date'] = non_normalized_columns['Date']
merged_df['Confirmed'] = non_normalized_columns['Confirmed']
merged_df['Deaths'] = non_normalized_columns['Deaths']

# Re-ordering
merged_df_cols = merged_df.columns.tolist()
cols = merged_df_cols[-4:] + merged_df_cols[:len(merged_df_cols)-4]

merged_df = merged_df[cols]

#### Converting states to one hot vector

In [12]:
one_hot_states = one_hot_encode(merged_df, 'Province_State')
one_hot_states_df = pd.DataFrame(columns=['State_'+x for x in state_names], data=one_hot_states)
merged_df = pd.concat([one_hot_states_df, merged_df], axis=1, sort=False)
merged_df.drop(['Province_State'], axis=1, inplace=True)

#### Create Day and Month

In [13]:
dates = merged_df['Date'].apply(lambda x: x.split('-')[0:2])
merged_df['Month'] = [int(x[0]) for x in dates]
merged_df['Day'] = [int(x[1]) for x in dates]

# Normalize Day and Month between 0 and 1
merged_df['Day'] = normalize(merged_df, 'Day')

# Exceptional case where we want the max month to be 12 even though this does not show up in the training data
max_ = 12
min_ = min(merged_df['Month']) #4
merged_df['Month'] = merged_df['Month'].apply(lambda x: (x-min_)/(max_-min_))

# Drop the date column
merged_df.drop(['Date'], inplace=True, axis=1)

In [14]:
merged_df

,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,State_California,State_Colorado,State_Connecticut,State_Delaware,State_Florida,State_Georgia,...,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Month,Day
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.144582,-0.148633,-0.156990,-0.150194,-0.149977,-0.160389,-0.149111,-0.158190,0.0,0.366667
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.143976,-0.148079,-0.157352,-0.149995,-0.149988,-0.160083,-0.149022,-0.157914,0.0,0.400000
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.144111,-0.148148,-0.157171,-0.150255,-0.150046,-0.160389,-0.148865,-0.156811,0.0,0.433333
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.143980,-0.148033,-0.157533,-0.150324,-0.149966,-0.160160,-0.149167,-0.157500,0.0,0.466667
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.144028,-0.147433,-0.158438,-0.150437,-0.149977,-0.160006,-0.149089,-0.156121,0.0,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.145325,-0.129430,-0.156809,-0.152003,-0.145456,-0.160696,-0.147614,7.236277,0.5,0.866667
7096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.145341,-0.123984,-0.155904,-0.151847,-0.145491,-0.160811,-0.147648,7.231174,0.5,0.900000
7097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.145352,-0.122876,-0.155180,-0.152020,-0.146130,-0.160773,-0.147837,6.418884,0.5,0.933333
7098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.145490,-0.128369,-0.158076,-0.151986,-0.146655,-0.160619,-0.148251,6.144580,0.5,0.966667


In [15]:
# Save Merged DataFrame
merged_df.to_csv('./data/merged_transformed.csv', index=False)